# CentraleSupelec - Natural language processing
# Practical session n°7

## Natural Language Inferencing (NLI): 

(NLI) is a classical NLP (Natural Language Processing) problem that involves taking two sentences (the premise and the hypothesis ), and deciding how they are related- if the premise *entails* the hypothesis, *contradicts* it, or *neither*.

Ex: 


| Premise | Label | Hypothesis |
| --- | --- | --- |
| A man inspects the uniform of a figure in some East Asian country. | contradiction | The man is sleeping. |
| An older and younger man smiling. | neutral | Two men are smiling and laughing at the cats playing on the floor. |
| A soccer game with multiple males playing. | entailment | Some men are playing a sport. |

### Stanford NLI (SNLI) corpus

In this labwork, I propose to use the Stanford NLI (SNLI) corpus ( https://nlp.stanford.edu/projects/snli/ ), available in the *Datasets* library by Huggingface.

    from datasets import load_dataset
    snli = load_dataset("snli")
    #Removing sentence pairs with no label (-1)
    snli = snli.filter(lambda example: example['label'] != -1) 

## Subject

You are asked to provide an operational Jupyter notebook that performs the task of NLI. For that, you need to tackle the following aspects of the problem:

1. Loading and preprocessing the data
2. Designing a PyTorch model that, given two sentences, decides how they are related (*entails*, *contradicts* or *neither*.)
3. Training and evaluating the model using appropriate metrics
4. (Optional) Allowing to play with the model (forward user sentences and visualize the prediction easily)
5. (Optional) Providing visual insight about the model (i.e. visualizing the attention if your model is using attention)

Although it is not mandatory, I suggest that you use a transformer model to perform the task. For that, you can use the *Transformer* library by Huggingface.

## Evaluation

The evaluation will be based on several criteria:

- Clarity and readability of the notebook. The notebook is the report of you project. Make it easy and pleasant to read.
- Justification of implementation choices (i.e. the network, the cost funtion, the optimizer, ...)
- Quality of the code. The various deeplearning and NLP labworks provide many example of good practices for designing experiments with neural networks. Use them as inspirational examples!

## Additional recommendations

- You are not seeking to publish a research paper! I'm not expecting state-of-the-art results! The idea of this labwork is to assess that you have integrated the skills necessary to handle textual data using deep neural network techniques.

- This labwork will be evaluated but we are still here to help you! Don't hesitate to request our help if you are stuck.

- If you intend to use BERT based models, let me give you an advice. The bert-base-* models available in *Transformers* need more than 12Go to be fine-tuned on GPU. To avoid memory issues, you can use several solutions: 

    - Use a lighter BERT based model such as DistilBERT, ALBERT, ...
    - Train a classification model on top of BERT, whithout fine-tuning it (i.e. freezing BERT weights)

## Huggingface documentations

In case you want to use the huggingface *Datasets* and *Transformer* libraries (which I advice), here are some useful documentation pages:

- Dataset quick tour

    https://huggingface.co/docs/datasets/quicktour.html
    
- Documentation on data preprocessing for transformers

    https://huggingface.co/transformers/preprocessing.html
    
- Transformer Quick tour (with distilbert example for classification).

    https://huggingface.co/transformers/quicktour.html
    


### Part 1 : Loading and preprocessing the data ###

Dans cette première partie, on va télécharger le dataset et le stocker dans un dataframe pandas. Ensuite, on effectura un preprocessing des données en effectuant une tokenization des phrases des corpus.

In [1]:
from datasets import load_dataset
snli = load_dataset("snli")
#Removing sentence pairs with no label (-1)
snli = snli.filter(lambda example: example['label'] != -1)


Dataset snli downloaded and prepared to /usr/users/gpusdi1/gpusdi1_26/.cache/huggingface/datasets/snli/plain_text/1.0.0/bb1102591c6230bd78813e229d5dd4c7fbf4fc478cec28f298761eb69e5b537c. Subsequent calls will reuse this data.


In [37]:
train = snli['train']
validation = snli['validation']
test = snli['test']

In [56]:
print(train)

Dataset({
    features: ['premise', 'hypothesis', 'label'],
    num_rows: 549367
})


In [42]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [71]:
from nltk.tokenize import word_tokenize 
import pandas as pd
import os

In [50]:
tokenizer(train['premise'],padding=True,truncation=True, return_tensors="pt")

{'input_ids': tensor([[  101,   138,  1825,  1113,   170,  3241, 15457,  1166,   170,  3088,
          1205, 15478,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [67]:
def encode(exemples):
    return(tokenizer(exemples['premise'],exemples['hypothesis'],padding=True,truncation=True))

In [72]:
train = train.map(encode, batched=True,num_proc=os.cpu_count())
test = test.map(encode, batched=True,num_proc=os.cpu_count())
validation = validation.map(encode, batched=True,num_proc=os.cpu_count())

In [76]:
print(train['premise'][0])
print(tokenizer.decode(train['input_ids'][0]))

A person on a horse jumps over a broken down airplane.
[CLS] A person on a horse jumps over a broken down airplane. [SEP] A person is training his horse for a competition. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [39]:
def tokenize_to_dataframe(dataset):
    data = {'premise' : [],'premise_tokenize' : [],'hypothesis' : [],'hypothesis_tokenize' : []}
    for sentence in dataset['premise']:
        data['premise_tokenize'].append(tokenizer(sentence,padding=True, 
                                                  truncation=True, return_tensors="pt"))
    for hypothesis in dataset['hypothesis']:
        data['hypothesis_tokenize'].append(tokenizer(hypothesis,padding=True, 
                                                     truncation=True, return_tensors="pt"))
    data['label'] = dataset['label']
    data['premise'] = dataset['premise']
    data['hypothesis'] = dataset['hypothesis']
    return(pd.DataFrame(data=data))
        

In [40]:
df_train = tokenize_to_dataframe(train)
print('Les données de train sont tokenizer.')
df_test = tokenize_to_dataframe(test)
print('Les données de test sont tokenizer.')
df_validation = tokenize_to_dataframe(validation)
print('Les données de validation sont tokenizer.')

Les données de train sont tokenizer.
Les données de test sont tokenizer.
Les données de validation sont tokenizer.


On vérifie la forme de notre dataset :

In [36]:
df_train.head()

,premise,hypothesis,label
0,"[<start>, A, person, on, a, horse, jumps, over...","[<start>, A, person, is, training, his, horse,...",1
1,"[<start>, A, person, on, a, horse, jumps, over...","[<start>, A, person, is, at, a, diner, ,, orde...",2
2,"[<start>, A, person, on, a, horse, jumps, over...","[<start>, A, person, is, outdoors, ,, on, a, h...",0
3,"[<start>, Children, smiling, and, waving, at, ...","[<start>, They, are, smiling, at, their, paren...",1
4,"[<start>, Children, smiling, and, waving, at, ...","[<start>, There, are, children, present, <end>]",0


Les données sont stockées dans un dataframe et tokeniser. On peut passer à l'étape de création d'un modèle

### Part 2 : Creation of a model ###

In [78]:
from transformers import DistilBertConfig, DistilBertTokenizer, DistilBertForSequenceClassification

In [79]:
model_name = "distilbert-base-uncased"
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=3)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

In [ ]:
type(model)